<a href="https://colab.research.google.com/github/Bakir-Bhuyain/prescription_analyzer_model/blob/main/Copy_of_TROCR_TEXT_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python transformers torch torchvision pillow gradio

In [ ]:
import cv2
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import gradio as gr
import io
import requests

In [ ]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
image_path = "https://i.ibb.co.com/gLhQLND5/Whats-App-Image-2025-11-05-at-18-50-31-41832d9b.jpg"
image = Image.open(requests.get(image_path, stream=True).raw).convert("RGB")

In [ ]:
image_np = np.array(image)

In [ ]:
gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (100, 5))
dilate = cv2.dilate(thresh, kernel, iterations=2)

contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])

In [ ]:
all_lines_text = []

for contour in contours:
  x, y, w, h = cv2.boundingRect(contour)

  if h > 20 and w > 50:
    cropped = image_np[y:y+h, x:x+w]
    pil_img = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

    pixel_values = processor(images=pil_img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    all_lines_text.append(text)
    # print(f"Detected Line: {text}")

final_text = "\n".join(all_lines_text)

print("Final Extracted Prescription Text:")
print(final_text)

Final Extracted Prescription Text:
This is my handwriting


In [ ]:
def extract_text_from_image(image_path):


    # image_path = "https://i.ibb.co.com/W4kR64Vd/img-8.jpg"
    image = Image.open(requests.get(image_path, stream=True).raw).convert("RGB")

    image_np = np.array(image)

    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (100, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])

    all_lines_text = []

    for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)

      if h > 20 and w > 50:
        cropped = image_np[y:y+h, x:x+w]
        pil_img = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

        pixel_values = processor(images=pil_img, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)
        text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        all_lines_text.append(text)
        # print(f"Detected Line: {text}")

    final_text = "\n".join(all_lines_text)

    # print("Final Extracted Prescription Text:")
    # print(final_text)
    return final_text

iface = gr.Interface(
    fn=extract_text_from_image,
    inputs=gr.Textbox(label="Image URL"),
    # inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Textbox(label="Extracted Text"),
    title="Multiline Handwritten OCR",
    description="Upload an image URL containing handwritten text to extract the text."
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://45b72fc4509c236917.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
